In [1]:
import pandas as pd
from datetime import datetime as dt
from concurrent.futures import ProcessPoolExecutor

import warnings as wn
wn.filterwarnings("ignore")

In [2]:
base_url = "https://www.marketwatch.com/investing/stock/"

In [3]:
def get_intrinsic_value(ticker):
    try:
        analyst_estimates = pd.read_html(f"{base_url}{ticker}/analystestimates", index_col=0)
        eps_estimates = analyst_estimates[6]
        eps_estimates = eps_estimates.loc["Average", eps_estimates.columns.astype(int) >= dt.today().year]

        ev_ebitda = pd.read_html(f"{base_url}{ticker}/company-profile")[4][1][6]
        eps = eps_estimates.append(pd.Series({eps_estimates.index[-1]: eps_estimates[-1] * ev_ebitda}))

        discount_rate = eps_estimates.pct_change().mean()

        fy_report_date = analyst_estimates[4][1]["FY Report Date"]
        fy_report_date = pd.to_datetime(fy_report_date) + pd.offsets.MonthEnd()
        discount_factor = (fy_report_date.date() - dt.today().date()).days / 365
        discount_factor = [discount_factor + i for i in range(3)]
        discount_factor += [discount_factor[-1]]

        pv = eps / (1 + discount_rate) ** discount_factor
        fair_value = sum(pv)

        last_close = pd.read_html(f"{base_url}{ticker}")[1]["Close"][0]

        data = pd.DataFrame(
                    data={"EV/EBITDA": ev_ebitda,
                        "Discount Rate": f"{(discount_rate * 100):.2f}%",
                        "Last Close": last_close,
                        "Fair Value": f"${fair_value:.2f}"},
                    index=[ticker]
                    )
        return data
    except:
        pass


def multiprocessing(tickers):
    with ProcessPoolExecutor() as executor:
        df = pd.concat(list(executor.map(get_intrinsic_value, tickers)), axis=0)
    return df

In [4]:
tickers = ["COST", "MA", "V", "SPGI", "MCO", "UNH", "LLY", "MRK", "NVO", "AAPL",
           "MSFT", "ASML", "AVGO", "ODFL", "PG", "JNJ", "PEP", "KO", "GOOG", "BRK.B"]

In [5]:
df = multiprocessing(tickers)
df

,EV/EBITDA,Discount Rate,Last Close,Fair Value
COST,23.34,8.64%,$538.00,$372.92
MA,28.48,19.21%,$400.69,$353.15
V,24.03,14.38%,$242.21,$225.75
SPGI,30.88,14.50%,$406.56,$399.75
MCO,29.32,14.93%,$347.21,$299.98
UNH,14.77,12.95%,$451.70,$416.32
LLY,43.98,36.09%,$440.60,$353.40
MRK,12.58,17.32%,$107.47,$101.17
NVO,28.68,16.33%,$154.75,$152.88
AAPL,23.31,10.02%,$189.77,$154.39


In [6]:
df.to_csv("marketwatch-fair-values.csv", encoding="utf-8-sig")